## Intro

It is common to have a remote and dispersed team nowadays. Therefore, It is challenging to have a clear picture of where your team is.

GitHub provides some handy data to help us understand the team's workload and progress.

StackQL has an official GitHub provider. With ```pystackql```, we can use ```Jupyter Notebook``` to retrieve data from GitHub with SQL, drastically improving the code's readability.

We can then store the data in a data warehouse solution to produce better analytics. Here our choice is ```BigQuery```.

## Set up the dependencies and configurations

###  Assumption: 
1. you have downloaded stackql and is in your system path; Follow [this instruction to install](https://stackql.io/downloads)
2. you have cloned [```pystackql```](https://github.com/stackql/pystackql) repo in the project directory
3. you have created your GitHub Person Access Token and saved it as environment variable GITHUB_TOKEN

In [2]:
# Import dependencies
from pystackql import StackQL
import json 
import os
import pandas as pd
import base64
from IPython.display import display

### Set up authentication

In [19]:
github_token= 'your github token'

## it needs to be base64 encoded
github_creds = base64.b64encode(bytes(github_token, encoding='utf-8')).decode()
os.environ['GITHUB_CREDS'] = github_creds
auth = { 
    "github": 
      { "type": "basic", "credentialsenvvar": "GITHUB_CREDS" }
}
iql = StackQL(auth=json.dumps(auth))

To make the script easy to duplicate in any environment, we will use pystackql to pull the newest version of github provider from registry

In [4]:
## pull github registry
registry_res = iql.execute('REGISTRY LIST;')
registry_list = json.loads(registry_res)
github_registry = list(filter(lambda reg: reg["provider"] == 'github', registry_list))[0]
github_version = github_registry['version']

pull_provider_query = """
REGISTRY PULL github %s;
""" % github_version

res = iql.execute(pull_provider_query)


## Using Github provider
We will use [Intel open source DAOS project](https://github.com/daos-stack/daos) as example 

We will store the response in pandas dataframe

In [36]:
def get_dataframe_from_query(query: str) -> pd.DataFrame :
    res = iql.execute(query)
    try: 
        res_obj = json.loads(res)
        if "error" in res_obj:
            raise AttributeError()
        if not res_obj:
            raise TypeError(res)
        data = pd.DataFrame.from_dict(res_obj)
        return data
    except Exception as error:
        if not isinstance(error, TypeError):
            error.args = ('StackQL execute error with error: %s, res: %s, error type: %s, res type: %s' %(error, res, type(error), type(res)), *error.args)
        raise error


### Overview
First lets look at what we can get from the Github provider

In [37]:
def show_resource():
    query = 'show resources in github.repos'
    data = get_dataframe_from_query(query)
    print(data.head(5))

show_resource()


                                     id                     name
0      github.repos.access_restrictions      access_restrictions
1  github.repos.admin_branch_protection  admin_branch_protection
2               github.repos.app_access               app_access
3                github.repos.autolinks                autolinks
4        github.repos.branch_protection        branch_protection


Find the complete list [here](https://registry.stackql.io/providers/github/)

In [1]:
test_repo = 'daos'
test_owner = 'daos-stack'

We will need to extract the user name of commit author from url

In [12]:
def get_username_from_url(url):
    try:
        if url != None and url != 'null' and len(url):
            return url.split('/users/')[1]
        return None
    except:
        print('get user name from url error with url %s', url)
def get_username_from_column(list_data: list):
    return list(map(lambda url: get_username_from_url(url), list_data));

### Get the contributors of the repo

In [14]:
### get the developers
#github.repos.contributors
def get_contributors_of_repo(repo, owner):
    query = """
    SELECT name, email, id, type, url
    FROM github.repos.contributors
    where repo = '%s' AND owner = '%s';
    """ % (repo, owner)
    print(query)
    data = get_dataframe_from_query(query)
    return data
contributor_data = get_contributors_of_repo(test_repo, test_owner)
contributor_data['username'] = get_username_from_column(contributor_data['url'].to_list())


    SELECT name, email, id, type, url
    FROM github.repos.contributors
    where repo = 'daos' AND owner = 'daos-stack';
    
  email        id  name  type                                      url  \
0  null  10464486  null  User  https://api.github.com/users/jolivier23   

     username  
0  jolivier23  


### explore commit activity 

In [38]:

def get_commits(repo, owner) :
    query = """
    SELECT 
    JSON_EXTRACT(commit, '$.message') as message, 
    sha, 
    JSON_EXTRACT(author, '$.url') as url,
    JSON_EXTRACT(commit, '$.author.date') as commit_date
    FROM github.repos.commits 
    where repo = '%s' AND owner = '%s';
    """ % (repo, owner)
    data = get_dataframe_from_query(query)
    # commits_json = data['commit'].to_list()
    # commits = list(map(lambda commit: json.loads(commit), commits_json)) 
    # data['commit'] = commits   
    return data
commits = get_commits(test_repo, test_owner)
commits['username'] = get_username_from_column(commits['url'].to_list())




In [40]:
display(commits.head(1))

,commit_date,message,sha,url,username
0,2022-07-21T20:16:56Z,DAOS-11155 test: Increase pool size for contai...,f8be4316cf58c90abc409027eb52dc0e852d24c9,https://api.github.com/users/shimizukko,shimizukko


### Other interesting resources in the repo that can reveal developer's activity

In [41]:
def get_pull_requests(repo, owner):
    #github.pulls.pull_requests
    query = """
    SELECT 
    number as pull_number, 
    JSON_EXTRACT(assignee, '$.url') as assignee_url, 
    JSON_EXTRACT(user, '$.url') as user_url, 
    state,
    updated_at
    FROM github.pulls.pull_requests
    where repo = '%s' AND owner = '%s'
    """ % (repo, owner)
    pull_requests = get_dataframe_from_query(query)
    pull_requests['assignee_username'] =get_username_from_column(pull_requests['assignee_url'].to_list())
    pull_requests['username'] =get_username_from_column(pull_requests['user_url'].to_list())
    return pull_requests;
pull_requests = get_pull_requests(test_repo, test_owner)
display(pull_requests.head(1))


,assignee_url,pull_number,state,updated_at,user_url,assignee_username,username
0,null,9777,open,2022-07-21T20:42:03Z,https://api.github.com/users/phender,None,phender


Get review from a single pull request

In [46]:

def get_pull_request_reviews(pull_number, repo, owner):
    #github.pulls.reviews
    query = """
    SELECT JSON_EXTRACT(user, '$.url') as user_url, state, body, submitted_at
    FROM github.pulls.reviews
    where repo = '%s' AND owner = '%s' AND pull_number = %s
    """ % (repo, owner, pull_number)
    try:
        pull_requests_reviews = get_dataframe_from_query(query)
        pull_requests_reviews['username'] =get_username_from_column(pull_requests_reviews['user_url'].to_list())
        return pull_requests_reviews;
    except(TypeError):
        raise TypeError        


In [47]:
display(get_pull_request_reviews(9656, test_repo, test_owner).head(1))

,body,state,submitted_at,user_url,username
0,LGTM. No errors found by checkpatch.,COMMENTED,2022-07-11T11:56:15Z,https://api.github.com/users/daosbuild1,daosbuild1


In [48]:
def get_reviews_for_pull_requests(pull_numbers: list, repo, owner, limit=100):
    ##loop pull requests data
    reviews_frames = []
    pull_numbers = pull_numbers[0:limit]
    for pull_number in pull_numbers:
        try:
            print('getting reviews for pull number %s' % pull_number)
            reviews = get_pull_request_reviews(pull_number, repo, owner)
            if reviews is not None and not reviews.empty:
                reviews_frames.append(reviews)
        except Exception as error:
            print(error)
            if isinstance(error, TypeError):
                continue
            else:
                return;
            
    return pd.concat(reviews_frames);

reviews_data = get_reviews_for_pull_requests(pull_requests['pull_number'].to_list(), test_repo, test_owner, limit=100)


getting reviews for pull number 9777
getting reviews for pull number 9775
getting reviews for pull number 9773
getting reviews for pull number 9772
getting reviews for pull number 9771
getting reviews for pull number 9770
getting reviews for pull number 9769
getting reviews for pull number 9768
getting reviews for pull number 9767
getting reviews for pull number 9766

getting reviews for pull number 9764
getting reviews for pull number 9763
getting reviews for pull number 9762
getting reviews for pull number 9761
getting reviews for pull number 9760
getting reviews for pull number 9757

getting reviews for pull number 9755
getting reviews for pull number 9754
getting reviews for pull number 9752
getting reviews for pull number 9746
getting reviews for pull number 9745

getting reviews for pull number 9744
getting reviews for pull number 9743
getting reviews for pull number 9742
getting reviews for pull number 9741
getting reviews for pull number 9740
getting reviews for pull number 973

In [50]:
reviews = reviews_data.copy()
reviews['username'] = get_username_from_column(reviews['user_url'].to_list())
display(reviews.head(5))

,body,state,submitted_at,user_url,username
0,Style warning(s) for job https://build.hpdd.in...,CHANGES_REQUESTED,2022-07-21T20:36:16Z,https://api.github.com/users/daosbuild1,daosbuild1
0,LGTM. No errors found by checkpatch.,COMMENTED,2022-07-21T14:34:52Z,https://api.github.com/users/daosbuild1,daosbuild1
0,LGTM. No errors found by checkpatch.,COMMENTED,2022-07-21T11:26:37Z,https://api.github.com/users/daosbuild1,daosbuild1
1,,APPROVED,2022-07-21T15:48:29Z,https://api.github.com/users/daltonbohning,daltonbohning
0,Style warning(s) for job https://build.hpdd.in...,DISMISSED,2022-07-21T09:56:30Z,https://api.github.com/users/daosbuild1,daosbuild1


## Aggregate the activities by contributor
Note: to improve the database efficiency, here we choose to de-normalize the data and nest them together

In [64]:
def aggregate_user_activity (row, commits: pd.DataFrame,  reviews: pd.DataFrame, pull_requests: pd.DataFrame):
    username = row['username']
    row['commits'] = commits[commits['username'] == username]
    row['pull_requests'] = pull_requests[pull_requests['username'] == username]
    row['reviews'] = reviews[reviews['username'] == username]
    return row

def get_user_activity(users: pd.DataFrame, commits: pd.DataFrame,  reviews: pd.DataFrame, pull_requests: pd.DataFrame, owner, repo):
    #avoid rate limit
    activities = users.apply(aggregate_user_activity, args=(commits, reviews, pull_requests), axis=1)
    activities['owner'] = owner;
    activities['repo'] = repo
    return activities;

activities = get_user_activity(contributor_data, commits, reviews, pull_requests, test_owner, test_repo)

In [54]:
display(activities.head(1).columns)

Index(['email', 'id', 'name', 'type', 'url', 'username', 'commits',
       'pull_requests', 'reviews', 'owner', 'repo'],
      dtype='object')

### Save the dataframe as a json file
Uploading pandas dataframe to BigQuery directly is a bit of headache, so lets save the data retrieved as JSON file first

In [65]:
activities.to_json('user-activities.json', orient='records', lines=True) ##save as new line delimited json

# Upload data to Google BigQuery
First, Create and download a key file follow this [link](https://cloud.google.com/docs/authentication/getting-started) 

In [67]:
from google.cloud import bigquery

PROJECT='storyscore-356114' #replace with your project id
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service-key.json' ##replace it with path to your key file


def get_table_ref(client: bigquery.Client, dataset_id, table_id):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    return table_ref


def create_load_job_config(job_type='WRITE_TRUNCATE'):
    return bigquery.LoadJobConfig(
    write_disposition=job_type,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True
    )

def upload_json(filename: str, dataset_id: str, table_id: str, config):
    print('uploading table: %s'%(table_id))
    client = bigquery.Client(project=PROJECT)
    table_ref = get_table_ref(client, dataset_id, table_id)
    with open(filename, "rb") as source_file:
        job = client.load_table_from_file(
            source_file, table_ref, job_config=config
        )  # Make an API request.
        job.result()  # Wait for the job to complete.;
    table = client.get_table(table_ref)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def upload_user_activity(filename: str):
    dataset = 'github_user_activity'
    table_name = 'user_activity'
    config = create_load_job_config()
    upload_json(filename=filename, dataset_id=dataset, table_id=table_name, config= config)
    return


In [68]:
upload_user_activity('./user-activities.json')

uploading table: user_activity
Loaded 85 rows and 11 columns to user_activity


### Check the table in BigQuery
Image 1

Number of pull requests by each contributor:
image 2